In [ ]:
%%configure -f
{"executorCores": 4, "executorMemory": "8096M", "conf": {"spark.default.parallelism": 1000}}

In [ ]:
# def get_aws_credentials():
#     """
#         Loads AWS credentials from credentials file and sets env variables
#     """
#     credentials_file = os.getenv("HOME") + '/.aws/credentials'

#     try:
#         with open(credentials_file, 'r') as f:
#             lines = f.read()
#         config = dict([tuple([kv.strip() for kv in line.split('=')]) for line in lines.split('\n') if '=' in line])
#     except FileNotFoundError:
#         config = dict()
#     if "aws_access_key_id" in config and "aws_secret_access_key" in config:
#         return config["aws_access_key_id"], config["aws_secret_access_key"]
#     elif "AWS_ACCESS_KEY_ID" in os.environ and "AWS_SECRET_ACCESS_KEY" in os.environ:
#         return os.environ["AWS_ACCESS_KEY_ID"], os.environ["AWS_SECRET_ACCESS_KEY"]
#     return None, None

In [ ]:
# import os
# from pyspark.sql import SparkSession

# # Set Spark Home
# os.environ["SPARK_HOME"] = "/Users/weikun.wang/dev/spark-2.4.3-bin-hadoop2.7"

# # Get AWS credentials
# aws_access_key_id, aws_secret_access_key = get_aws_credentials()

# # Create Spark Session
# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName("SparkTest") \
#     .config("spark.driver.host", '127.0.0.1') \
#     .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
#     .config("spark.hadoop.fs.AbstractFileSystem.s3.impl", "org.apache.hadoop.fs.s3a.S3A") \
#     .config("spark.hadoop.fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A") \
#     .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
#     .config("spark.hadoop.fs.s3.endpoint", "s3.amazonaws.com") \
#     .config("spark.executor.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
#     .config("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
#     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#     .config("spark.driver.memory", "6g") \
#     .config("spark.hadoop.fs.s3.access.key", aws_access_key_id) \
#     .config("spark.hadoop.fs.s3.secret.key", aws_secret_access_key) \
#     .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
#     .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
#     .getOrCreate()
# spark

In [ ]:
# import pandas as pd

# import math
# import gensim
# import random
# import numpy as np
# from gensim.utils import tokenize
# import string
# from unidecode import unidecode
# from langdetect import detect
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType, BooleanType
# from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
# from pyspark.sql.window import Window

# from pyspark.sql import SparkSession
# import pyspark.sql.functions as F
# import pyspark.sql.types as T

In [ ]:
import pandas as pd

import gensim
from unidecode import unidecode
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import string
import numpy as np
from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [ ]:
df = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")
df.printSchema()

In [ ]:
df = df.distinct()

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = unidecode(text)
    return text

normalize_text_udf = udf(normalize_text, StringType())

df = df.withColumn("name_normalized", normalize_text_udf("name"))

In [ ]:
# df.count()

In [ ]:
string_columns = [item[0] for item in df.dtypes if item[1].startswith('string') and 'id' not in item[0]]
double_columns = [item[0] for item in df.dtypes if item[1].startswith('double')]
integer_columns = [item[0] for item in df.dtypes if item[1].startswith('int') and 'id' not in item[0]]
list_columns = "id_capiq"

In [ ]:
groupByKey = ["name_normalized", "country_of_incorporation", "region"]

In [ ]:
exprs = [F.collect_list(colName).alias(colName) for colName in df.columns if colName not in groupByKey]

In [ ]:
df_grouped = df.groupBy(groupByKey).agg(*exprs)

In [ ]:
df_grouped.count()

In [ ]:
string_columns

In [ ]:
# df_grouped_small = df_grouped.limit(1000).cache()

In [ ]:
# [type(np.mean(x)) for x in pd_df['total_revenue'] if x]

In [ ]:
# x = df_grouped_small.select('entity_id').toPandas()

In [ ]:
def transform_string_fields(field):
    if isinstance(field, list):
        return ", ".join(field)
    else:
        return field
    
def transform_double_fields(field):
    if isinstance(field, list):
        return float(np.mean(field)) if len(field) > 0 else [0.0]
    else:
        return field
    
def transform_integer_fields(field):
    if isinstance(field, list):
        return int(np.mean(field)) if len(field) > 0 else 0.0
    else:
        return field
    
def transform_entity_id(field):
    if isinstance(field, list):
        return field[0]
    else:
        return field

transform_string_fields_udf = udf(transform_string_fields, StringType())
transform_double_fields_udf = udf(transform_double_fields, DoubleType())
transform_integer_fields_udf = udf(transform_integer_fields, IntegerType())
transform_entity_id_udf = udf(transform_entity_id, StringType())

for col_name in string_columns:
    df_grouped = df_grouped.withColumn(col_name, transform_string_fields_udf(col_name))
    
for col_name in double_columns:
    df_grouped = df_grouped.withColumn(col_name, transform_double_fields_udf(col_name))
    
for col_name in integer_columns:
    df_grouped = df_grouped.withColumn(col_name, transform_integer_fields_udf(col_name))
    
df_grouped = df_grouped.withColumn("entity_id", transform_entity_id_udf(F.col("entity_id")))

In [ ]:
# df_grouped.show()

In [ ]:
(df_grouped
 .write
 .parquet("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_deduped", mode="overwrite")
)

In [ ]:
df_new = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_data_denormalized_deduped")

In [ ]:
df_new.agg(F.max(F.length(df_new["short_description"]))).collect()

In [ ]:
df.groupBy("name_normalized", "country_of_incorporation", "region").agg(F.count("name_normalized").alias("name_count")).orderBy(F.desc("name_count")).show(100)

In [ ]:
df_new = df_new.withColumn("description_length",F.length(df_new["short_description"]))

In [ ]:
df_new.select(F.max(
    F.struct("description_length", *(x for x in df_new.columns if x != "description_length"))
)).printSchema()

In [ ]:
avg_len = {}
for column in string_columns:
    if column == "name_normalized":
        continue
    avg_len[column] = df_new.agg(F.mean(F.length(df_new[column]))).collect()

In [ ]:
avg_len

In [ ]:
avg_len_ori = {}
for column in string_columns:
    avg_len_ori[column] = df.agg(F.mean(F.length(df[column]))).collect()

In [ ]:
avg_len_ori